<a href="https://colab.research.google.com/github/theinshort/asl_recognition/blob/main/acl_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from datasets import sign_language
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential


In [ ]:

# Set random seeds for reproducibility
np.random.seed(5)
tf.random.set_seed(2)


In [ ]:

# Load pre-shuffled training and test datasets
(x_train, y_train), (x_test, y_test) = sign_language.load_data()


In [ ]:

# Visualize the training data
labels = ['A', 'B', 'C']
fig = plt.figure(figsize=(20, 5))
for i in range(36):
    ax = fig.add_subplot(3, 12, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_train[i]))
    ax.set_title(labels[y_train[i]])
plt.show()


In [ ]:

# Examine the dataset
num_A_train = np.sum(y_train == 0)
num_B_train = np.sum(y_train == 1)
num_C_train = np.sum(y_train == 2)
num_A_test = np.sum(y_test == 0)
num_B_test = np.sum(y_test == 1)
num_C_test = np.sum(y_test == 2)

print("Training set:")
print(f"\tA: {num_A_train}, B: {num_B_train}, C: {num_C_train}")
print("Test set:")
print(f"\tA: {num_A_test}, B: {num_B_test}, C: {num_C_test}")


In [ ]:

# One-hot encode the data
y_train_OH = np_utils.to_categorical(y_train, 3)
y_test_OH = np_utils.to_categorical(y_test, 3)


In [ ]:

# Define the model
model = Sequential([
    Conv2D(filters=5, kernel_size=5, padding='same', activation='relu', input_shape=(50, 50, 3)),
    MaxPooling2D(pool_size=4),
    Conv2D(filters=15, kernel_size=5, padding='same', activation='relu'),
    MaxPooling2D(pool_size=4),
    Flatten(),
    Dense(3, activation='softmax')
])


In [ ]:

# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:

# Train the model
hist = model.fit(x_train, y_train_OH, validation_split=0.2, epochs=2, batch_size=32)


In [ ]:

# Test the model
score = model.evaluate(x_test, y_test_OH, verbose=0)
print(f'Test accuracy: {score[1]}')


In [ ]:

# Visualize Errors
y_probs = model.predict(x_test)
y_preds = np.argmax(y_probs, axis=1)
bad_test_idxs = np.where(y_test != y_preds)[0]

fig = plt.figure(figsize=(25, 4))
for i, idx in enumerate(bad_test_idxs):
    ax = fig.add_subplot(2, np.ceil(len(bad_test_idxs) / 2), i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_test[idx]))
    ax.set_title(f"{labels[y_test[idx]]} (pred: {labels[y_preds[idx]]})")
plt.show()